# Setup

In [1]:
import math
import pandas as pd
import numpy as np
import numpy.linalg as nplinalg
from scipy.spatial import distance

def hyperplane_hash(u, p):
    """Calculates the hyperplane hash value for p given the random vector u."""
    return int(np.dot(u, p) > 0)

def crosspolytope_hash(A, p):
    """Calculates the cross-polytope hash value for p given the rotation matrix A."""
    v_rotated = np.matmul(A, np.array([[1, 0, -1, 0], [0, 1, 0, -1]]))
    min_dist, min_dist_index = np.Inf, -1
    for i in range(4):
        v_i = np.array([v_rotated[0][i],v_rotated[1][i]])
        if nplinalg.norm(v_i-p)**2 < min_dist:
            min_dist = nplinalg.norm(v_i-p)**2
            min_dist_index = i
    return min_dist_index

# Example: Hyperplane LSH (p. 22)

Let us consider the points P = {p1, p2} and q in d=2:

In [2]:
p1 = np.array([math.cos((14/180)*math.pi), math.sin((14/180)*math.pi)])
p2 = np.array([math.cos((175/180)*math.pi), math.sin((175/180)*math.pi)])
q = np.array([math.cos((27/180)*math.pi), math.sin((27/180)*math.pi)])
print(f"p1: {p1}, p2: {p2}, q: {q}")

p1: [0.97029573 0.2419219 ], p2: [-0.9961947   0.08715574], q: [0.89100652 0.4539905 ]


The actual cosine similairities are:

In [3]:
cos_q_p1 = 1 - distance.cosine(q,p1)
cos_q_p2 = 1 - distance.cosine(q,p2)
print(f"cos(theta(q,p1)): {cos_q_p1:.4f}, cos(theta(q,p2)): {cos_q_p2:.4f}")

cos(theta(q,p1)): 0.9744, cos(theta(q,p2)): -0.8480


Showing that p1 and 1 are very similar, and p2 and q quite dissimilar.

Now, suppose we have two random vectors:

In [4]:
u1 = np.array([math.cos((50/180)*math.pi), math.sin((50/180)*math.pi)])
u2 = np.array([math.cos((112/180)*math.pi), math.sin((112/180)*math.pi)])
print(f"u1: {u1}, u2: {u2}")

u1: [0.64278761 0.76604444], u2: [-0.37460659  0.92718385]


The hash values generated with H = {h_u1, h_u2) are:

In [5]:
print(f"h_u1(p1) = {hyperplane_hash(u1,p1)} (u1*p1 = {np.dot(u1,p1):.4f})")
print(f"h_u1(p2) = {hyperplane_hash(u1,p2)} (u1*p2 = {np.dot(u1,p2):.4f})")
print(f"h_u1(q) = {hyperplane_hash(u1,q)} (u1*q = {np.dot(u1,q):.4f})")
print(f"h_u2(p1) = {hyperplane_hash(u2,p1)} (u2*p1 = {np.dot(u2,p1):.4f})")
print(f"h_u2(p2) = {hyperplane_hash(u2,p2)} (u2*p2 = {np.dot(u2,p2):.4f})")
print(f"h_u2(q) = {hyperplane_hash(u2,q)} (u2*q = {np.dot(u2,q):.4f})")

h_u1(p1) = 1 (u1*p1 = 0.8090)
h_u1(p2) = 0 (u1*p2 = -0.5736)
h_u1(q) = 1 (u1*q = 0.9205)
h_u2(p1) = 0 (u2*p1 = -0.1392)
h_u2(p2) = 1 (u2*p2 = 0.4540)
h_u2(q) = 1 (u2*q = 0.0872)


Concatenating h_u1 and h_u2 with an `AND`-construction would yield no Candidate Pair (CP), while using an `OR`-construction would make both p1 and p2 a CP with q. In practice, when using many more hash functions, the probability for random vectors like u1 (making q and p1 a CP) tends towards 167/180 and the one for vectors like u2 towards 32/180, thus being an *estimation* of the angle between the vectors, which corresponds to their cosine similarity.

# Example: Spherical LSH (p. 24)

Let the cross-polytope consist of the vertices:

In [6]:
v_tilde = np.array([[1, 0, -1, 0], [0, 1, 0, -1]])
print(v_tilde)

[[ 1  0 -1  0]
 [ 0  1  0 -1]]


Consider the LSH family H = {h_A1, h_A2} with

In [7]:
# Angle of rotation
theta_1, theta_2 = 50*math.pi/180, 158*math.pi/180
# Rotation matrices
A1 = np.array([[math.cos(theta_1), -math.sin(theta_1)], [math.sin(theta_1), math.cos(theta_1)]])
A2 = np.array([[math.cos(theta_2), -math.sin(theta_2)], [math.sin(theta_2), math.cos(theta_2)]])
print(A1)
print(A2)

[[ 0.64278761 -0.76604444]
 [ 0.76604444  0.64278761]]
[[-0.92718385 -0.37460659]
 [ 0.37460659 -0.92718385]]


Compute the vertices of the rotated polytopes:

In [8]:
v = np.matmul(A1, v_tilde)
v_dash = np.matmul(A2, v_tilde)
print(v)
print(v_dash)

[[ 0.64278761 -0.76604444 -0.64278761  0.76604444]
 [ 0.76604444  0.64278761 -0.76604444 -0.64278761]]
[[-0.92718385 -0.37460659  0.92718385  0.37460659]
 [ 0.37460659 -0.92718385 -0.37460659  0.92718385]]


The hash values generated with H are:

In [9]:
print(f"h_A1(p1) = {crosspolytope_hash(A1, p1)}")
print(f"h_A1(p2) = {crosspolytope_hash(A1, p2)}")
print(f"h_A1(q) = {crosspolytope_hash(A1, q)}")
print(f"h_A2(p1) = {crosspolytope_hash(A2, p1)}")
print(f"h_A2(p2) = {crosspolytope_hash(A2, p2)}")
print(f"h_A2(q) = {crosspolytope_hash(A2, q)}")

h_A1(p1) = 0
h_A1(p2) = 1
h_A1(q) = 0
h_A2(p1) = 2
h_A2(p2) = 0
h_A2(q) = 3


We see that again two similar items (q and p1) can have different hash values (see h_A2). But different from Hyperplane LSH, very dissimilar instances (q and p2) have no possibility to become a CP.